In [17]:
import pandas as pd
from functions import analysis, feature_engineering, train_test_split, create_model, cv, baseline, \
    test_model, test_baseline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
data = pd.read_csv("data.csv")
# выкинем колонки, которые не нужы для 1 го задания
data = data.drop(columns=['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 
                  'g0', 'g1', 'coeff_sum0', 'coeff_sum1'])
data.head()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,0,1,0,1,1


In [3]:
analysis(data)

Количество записей в таблице:  15821472
соотношние между классами:
1-ый класс: 2.6688351121817235 %, 2-ой класс 97.33116488781828 %

Nan в таблице:
False

Количество уникальных значений для os_id:  11
Количество уникальных значений для country_id:  17
Количество уникальных значений для impressions:  1
Количество уникальных значений для campaign_clicks:  822
Количество уникальных значений для banner_id:  1633


In [26]:
# Nan нет в таблице,поэтому пропуски заполнять не надо
# Так как значение этой колоники одинаково для всех строк, уберем ее
data = data.drop(columns=['impressions'])

#### Проанализируем чем отличаются случаи, когда были клики, от остальных

Посмотрим распределение по странам, операционным системам в случаях, когда был клик, и когда не было (у других фичей слишком много значений)

In [5]:
# по странам
countries_dist_click = pd.DataFrame(data.loc[data['clicks']==1]['country_id'].value_counts(normalize=True)).reset_index()
countries_dist_no_click = pd.DataFrame(data.loc[data['clicks']==0]['country_id'].value_counts(normalize=True)).reset_index()
countries_dist_no_click.merge(countries_dist_click, 'inner', 'index').rename({'country_id_x':'no_click_countries',
                                                                              'country_id_y':'click_countries' }, axis=1)
# По результатам не видно корреляцией между старной и обилием кликов за редкими исключениями вроде country_id=16
# Тем не менее информация о стране может хоть какой-нибудь скор докинуть: поэтому мы не будет ее выкидывать

,index,no_click_countries,click_countries
0,0,0.313052,0.321239
1,5,0.121233,0.102226
2,6,0.101050,0.063022
3,7,0.096413,0.096815
4,3,0.084440,0.087806
5,1,0.084344,0.112474
6,15,0.038368,0.029520
7,12,0.030311,0.026698
8,9,0.027631,0.031981
9,4,0.024601,0.023590


In [6]:
# по операционным системам
os_dist_click = pd.DataFrame(data.loc[data['clicks']==1]['os_id'].value_counts(normalize=True)).reset_index()
os_dist_no_click = pd.DataFrame(data.loc[data['clicks']==0]['os_id'].value_counts(normalize=True)).reset_index()
os_dist_no_click.merge(os_dist_click, 'inner', 'index').rename({'os_id_x':'no_click_os',
                                                                              'os_id_y':'click_os' }, axis=1)
# Эта фича куда более говорящая - на некоторых операционных системах люди кликают рекламу реже (os_id=2, 1), на других - чаще (os_id=3, 4, 5, 6)

,index,no_click_os,click_os
0,2,0.292751,0.193838
1,0,0.242925,0.274584
2,1,0.203629,0.101767
3,4,0.187886,0.282404
4,3,0.046968,0.086447
5,6,0.018872,0.046747
6,5,0.006882,0.014131
7,8,0.000081,0.000073
8,7,0.000006,0.000007


# Построение модели

In [19]:
# Построим бейзлайн с которым будем сравнивать
data = baseline(data)
test_baseline(data)

Log loss: 0.11234095165091723
Auc: 0.7323323967801383


In [20]:
# Добавим признаков
data = feature_engineering(data)
data.tail()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks,average_clicks,time,date,time_of_day,is_weekend,month,av_clicks_os,av_clicks_country,av_clicks_month,date_copy
15821467,2021-09-29 12:33:43,12,1520,0.0,1,9,1,0,0.0,12:33:43,2021-09-29,0.75,0,0,0.0,0.0,0.0,2021-09-29
15821468,2021-10-01 19:04:18,14,853,0.0,0,11,1,0,0.0,19:04:18,2021-10-01,0.25,0,1,0.0,0.0,0.0,2021-10-01
15821469,2021-09-28 08:34:49,30,649,0.0,4,0,1,0,0.0,08:34:49,2021-09-28,1.00,0,0,0.0,0.0,0.0,2021-09-28
15821470,2021-10-02 05:16:56,14,562,0.0,2,6,1,0,0.0,05:16:56,2021-10-02,0.00,1,1,0.0,0.0,0.0,2021-10-02
15821471,2021-10-01 04:41:34,14,1163,0.0,2,12,1,0,0.0,04:41:34,2021-10-01,0.00,0,1,0.0,0.0,0.0,2021-10-01


In [21]:
#Сделаем one-hot encoding фич. У фичей banner_id и zone_id слишком много значений: поэтому закодируем только
# самые часто встречающиеся значение (всего получится около 20 значений)

In [22]:
cats_banner = data['banner_id'].value_counts()[lambda x: x > 200000].index
cats_zone = data['zone_id'].value_counts()[lambda x: x > 200000].index
cats_country = data['country_id'].value_counts()[lambda x: x > 300000].index
other_cat_features =  ['os_id', 'date_copy']

zone_col = pd.get_dummies(pd.Categorical(data['zone_id'], categories=cats_zone)).add_prefix('zone_')
banner_col = pd.get_dummies(pd.Categorical(data['banner_id'], categories=cats_banner)).add_prefix('banner_')
country_col = pd.get_dummies(pd.Categorical(data['country_id'], categories=cats_country)).add_prefix('country_')
data = pd.get_dummies(data, columns= other_cat_features, drop_first=True)
data = pd.concat([data, zone_col],  axis=1)
data = pd.concat([data, banner_col], axis=1)

In [28]:
# Разобъем на тест и тест: в качестве теста возьмем последний день
features = [col for col in data.columns if col not in ['date_time', 'clicks', 'time', 'zone_id',
                                                       'banner_id', 'country_id']]
print(len(features))
X_train, Y_train, X_test, Y_test = train_test_split(data, features=features)
X_test = X_test.drop(columns=['date'])
X_train = X_train.drop(columns=['date'])


59


In [34]:
# Обрежем типы
float_col = ['av_clicks_os', 'av_clicks_country', 'average_clicks', 'campaign_clicks', 'av_clicks_month', 'time_of_day']
X_train.loc[:, float_col] = X_train.loc[:, float_col].astype('float16')

C:\Users\msk-lokteva\AppData\Local\Temp\ipykernel_13292\2013697066.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train.loc[:, float_col] = X_train.loc[:, float_col].astype('float16')


In [36]:
l2 = cv(X_train, Y_train, 'roc_auc')

L2: 0.01, score: [0.73562817 0.66590017 0.63570834]
L2: 0.1, score: [0.73969069 0.66833498 0.64718062]
L2: 1, score: [0.73695573 0.66870336 0.64901972]
L2: 10, score: [0.73530512 0.66874707 0.64911974]
L2: 100, score: [0.73425253 0.66875518 0.64910036]


In [40]:
# Возьмем l2=0.1

In [37]:
tuned_model = create_model(l2)
tuned_model.fit(X_train, Y_train)

LogisticRegression(C=0.1, solver='liblinear')

In [42]:
X_test.loc[:, float_col] = X_test.loc[:, float_col].astype('float16')
test_model(tuned_model,X_test, Y_test )

Log loss: 0.135430422031244
Auc: 0.7668656691087233


Полученная модель превосхолит бейзлайн по метрике Auc